# Segmenting and Clustering Neiborhoods in Toronto - Week 3 Assignment

### by **Jen Park**



## 1> Web Scraping 
#### (Data Source URL: *https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M*)


##### Task 1-1: Use pandas, or the BeautifulSoup package, or any other way you are comfortable with to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis

from bs4 import BeautifulSoup # library to handle requests

import requests

from sklearn.cluster import KMeans # import k-means from clustering stage

print('Libraries imported.')

Libraries imported.


In [2]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,'lxml')
table = soup.find_all('table')[0]
df = pd.read_html(str(table))
neighborhood = pd.DataFrame(df[0])

#### *Task 1-2: Create the dataframe as shown in the assignment instruction page.*

- Ignore cells with "**Not Assigned**" value

In [3]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [4]:
df

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


##### Task 1-3: In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.


In [5]:
df.shape

(103, 3)

## 2> Create a pandas dataframe with coordinates information

##### Task 2-1: Get the latitude and the longitude coordinates of each neighborhood. 

In [6]:
gdata = pd.read_csv("https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv")
gdata

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [7]:
gdata.shape

(103, 3)

##### Task 2-2: Use the csv file to create the dataframe shown in the assignment page.

In [8]:
# define the dataframe columns
column_names = ['Postal Code','Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [9]:
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude


In [10]:
# using merge function by setting how='inner'
neighborhoods = pd.merge(df, gdata, 
                   on='Postal Code', 
                   how='inner')
  
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto Business,Enclave of M4L,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## 3> Explore and cluster the neighborhoods in Toronto.

- Here, I will replicate the same analysis we did to the New York City data in the lab.

In [11]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 15 boroughs and 103 neighborhoods.


In [12]:
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

# All requested packages already installed.



In [13]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))

The coordinates of Toronto are 43.6534817, -79.3839347.


### Create a map of Toronto with neighborhoods superimposed on top.

In [14]:
import folium

In [15]:
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=9.5)

# adding markers to map
for latitude, longitude, borough, neighbourhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Define Foursquare Credentials and Version

In [16]:
CLIENT_ID = 'C0GOCIQAFJPDK44UVVINSC4W5WV2YH5EFKO1SG0MDBKYIXBP' # your Foursquare ID
CLIENT_SECRET = 'R00Q3Z5CNCQB01EBVIOCT5XQ4NUQXULYZUXSPMC0PGAZESL1' # your Foursquare Secret
VERSION = '20210612' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: C0GOCIQAFJPDK44UVVINSC4W5WV2YH5EFKO1SG0MDBKYIXBP
CLIENT_SECRET:R00Q3Z5CNCQB01EBVIOCT5XQ4NUQXULYZUXSPMC0PGAZESL1


### Let's create a function to explore all the neighborhoods in Toronto.

In [17]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius
            )
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',
                  'Venue Category']
    
    return(nearby_venues)

### Now write the code to run the above function on each neighborhood and create a new dataframe called *toronto_venues*.

In [18]:
toronto_venues = getNearbyVenues(names=neighborhoods['Neighborhood'], 
                                 latitudes=neighborhoods['Latitude'], 
                                 longitudes=neighborhoods['Longitude']
                                )


Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

### The size of the resulting dataframe is as follows:

In [19]:
print(toronto_venues.shape)
toronto_venues.head(50)

(1280, 5)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,Park
1,Parkwoods,43.753259,-79.329656,KFC,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,Variety Store,Food & Drink Shop
3,Parkwoods,43.753259,-79.329656,Corrosion Service Company Limited,Construction & Landscaping
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,Hockey Arena
5,Victoria Village,43.725882,-79.315572,Portugril,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,Tim Hortons,Coffee Shop
7,Victoria Village,43.725882,-79.315572,Pizza Nova,Pizza Place
8,"Regent Park, Harbourfront",43.654260,-79.360636,Tandem Coffee,Coffee Shop
9,"Regent Park, Harbourfront",43.654260,-79.360636,Roselle Desserts,Bakery


Let's check how many venues were returned for each neighborhood

In [20]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category
Neighborhood,,,,
Agincourt,4,4,4,4
"Alderwood, Long Branch",8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",18,18,18,18
Bayview Village,4,4,4,4
"Bedford Park, Lawrence Manor East",22,22,22,22
...,...,...,...,...
Willowdale West,5,5,5,5
"Willowdale, Newtonbrook",2,2,2,2
Woburn,3,3,3,3


#### Let's find out how many unique categories can be curated from all the returned venues

In [21]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 224 uniques categories.


## Analyze Each Neighborhood

In [22]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Aquarium,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
toronto_onehot.shape

(1280, 224)

### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [25]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Truck Stop,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.045455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,Willowdale West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
95,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [26]:
toronto_grouped.shape


(99, 224)

### Let's print each neighborhood along with the top 5 most common venues

In [27]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge  0.25
1             Breakfast Spot  0.25
2  Latin American Restaurant  0.25
3               Skating Rink  0.25
4                Yoga Studio  0.00


----Alderwood, Long Branch----
                venue  freq
0         Pizza Place  0.25
1      Sandwich Place  0.12
2            Pharmacy  0.12
3  Athletics & Sports  0.12
4         Coffee Shop  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
               venue  freq
0               Bank  0.11
1        Coffee Shop  0.11
2  Health Food Store  0.06
3              Diner  0.06
4        Pizza Place  0.06


----Bayview Village----
                 venue  freq
0                 Bank  0.25
1   Chinese Restaurant  0.25
2  Japanese Restaurant  0.25
3                 Café  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
            venue  freq
0  Sandwich Place  0.09
1      Restaurant  0.09
2     Coffee Shop  0.09
3   Women's

                        venue  freq
0  Construction & Landscaping   0.2
1          Photography Studio   0.2
2                        Park   0.2
3                 Swim School   0.2
4                    Bus Line   0.2


----Leaside----
                 venue  freq
0  Sporting Goods Shop  0.12
1          Coffee Shop  0.12
2     Sushi Restaurant  0.08
3                 Bank  0.08
4         Burger Joint  0.08


----Little Portugal, Trinity----
                     venue  freq
0                      Bar  0.10
1    Vietnamese Restaurant  0.07
2         Asian Restaurant  0.07
3  New American Restaurant  0.07
4              Yoga Studio  0.03


----Malvern, Rouge----
                      venue  freq
0      Fast Food Restaurant   1.0
1           Harbor / Marina   0.0
2            Medical Center   0.0
3  Mediterranean Restaurant   0.0
4             Metro Station   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
                      venue  freq
0                Playground  

                       venue  freq
0                Pizza Place  0.29
1             Sandwich Place  0.14
2                Coffee Shop  0.14
3  Middle Eastern Restaurant  0.14
4             Discount Store  0.14


----Wexford, Maryvale----
                       venue  freq
0  Middle Eastern Restaurant   0.4
1                Auto Garage   0.2
2                 Smoke Shop   0.2
3             Sandwich Place   0.2
4                Music Venue   0.0


----Willowdale South----
                 venue  freq
0          Coffee Shop  0.14
1     Ramen Restaurant  0.10
2          Pizza Place  0.07
3  Japanese Restaurant  0.07
4     Sushi Restaurant  0.07


----Willowdale West----
           venue  freq
0    Pizza Place   0.2
1  Grocery Store   0.2
2       Pharmacy   0.2
3    Coffee Shop   0.2
4    Supermarket   0.2


----Willowdale, Newtonbrook----
                      venue  freq
0                      Park   1.0
1               Yoga Studio   0.0
2               Music Venue   0.0
3  Mediterranean 